<a href="https://colab.research.google.com/github/H-jamesR2/c0gniz4nt-project-genAI/blob/main/_Z_RegularProjects/generative-ai-with-azure-machine-learning/Implementing_Cloud_Based_NLP_with_TransformersImplementing_Cloud_Based_NLP_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# download dependencies
!pip install transformers datasets torch numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install kagglehub

In [5]:
import os
import bz2
import urllib.request
import random
import torch
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [6]:
import kagglehub

In [7]:
# Set seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [8]:
# Download dataset using kagglehub
dataset_path = kagglehub.dataset_download("bittlingmayer/amazonreviews")

# Ensure the directory exists
os.makedirs("amazon_sentiment", exist_ok=True)

# Move downloaded files to desired location
train_file = os.path.join(dataset_path, "train.ft.txt.bz2")
test_file = os.path.join(dataset_path, "test.ft.txt.bz2")

os.rename(train_file, "amazon_sentiment/train.ft.txt.bz2")
os.rename(test_file, "amazon_sentiment/test.ft.txt.bz2")

print("Dataset downloaded and moved to 'amazon_sentiment/' folder.")

100%|██████████| 493M/493M [00:02<00:00, 194MB/s]

Extracting files...


Dataset downloaded and moved to 'amazon_sentiment/' folder.


In [9]:
def extract_bz2(file_path):
    with bz2.BZ2File(file_path, "r") as f:
        return [line.decode("utf-8").strip() for line in f]

# Extract train & test data
train_data = extract_bz2("amazon_sentiment/train.ft.txt.bz2")
test_data = extract_bz2("amazon_sentiment/test.ft.txt.bz2")

In [10]:
def parse_fasttext_format(data):
    parsed_data = []

    for line in data:
        parts = line.split(" ", 1)  # Split into label and text
        if len(parts) != 2:
            continue  # Skip invalid lines
        label, text = parts
        label = 0 if label == "__label__1" else 1  # Convert labels to 0/1
        parsed_data.append({"text": text, "label": label})

    return parsed_data

# Convert train/test data
train_data_parsed = parse_fasttext_format(train_data)
test_data_parsed = parse_fasttext_format(test_data)

In [17]:
# Function to balance and downsample
def balance_dataset(data, target_size):
    # Separate positive and negative samples
    positive_samples = [item for item in data if item['label'] == 1]
    negative_samples = [item for item in data if item['label'] == 0]

    print(f"Original positive samples: {len(positive_samples)}")
    print(f"Original negative samples: {len(negative_samples)}")

    # Target half of the size for each class
    half_size = target_size // 2

    # Ensure we don’t exceed available samples
    num_positive = min(half_size, len(positive_samples))
    num_negative = min(half_size, len(negative_samples))

    # Randomly sample from each class
    balanced_positive = random.sample(positive_samples, num_positive)
    balanced_negative = random.sample(negative_samples, num_negative)

    # Combine and shuffle
    balanced_data = balanced_positive + balanced_negative
    random.shuffle(balanced_data)

    return balanced_data

# Apply to train and test data
train_target_size = 18000
test_target_size = 2000

balanced_train_data = balance_dataset(train_data_parsed, train_target_size)
balanced_test_data = balance_dataset(test_data_parsed, test_target_size)

Original positive samples: 1800000
Original negative samples: 1800000
Original positive samples: 200000
Original negative samples: 200000


In [18]:
# Convert to Dataset
train_dataset = Dataset.from_list(balanced_train_data)
test_dataset = Dataset.from_list(balanced_test_data)

#train_dataset = Dataset.from_list(train_data_parsed)
#test_dataset = Dataset.from_list(test_data_parsed)

# Quick check
print(train_dataset[0])

{'text': "Facts in spots are wrong, brings up credibility issue: I bought this book with high hopes, being a teacher, and an avid video game player. When I got the book, I immediately turned to the chapter which talked about Everquest (having played Everquest since its beginning until late last year). Imagine my surprise when I found an extreme amount of errors in how he describes things that happen in the game. For example, he says that when a player dies, the corpse drops to the ground spilling all the objects it was carrying and the player comes back in a weaker form with worse armor. This never happened even in the early stages of Everquest. Besides, the publication date is 2003 regardless.That was one such example of many I can cite from that chapter, but that brings up credibility issues for me in reading any further in the book. If that chapter's facts are wrong, then how do I know that other facts in other chapters are correct?", 'label': 0}


In [19]:
# Check balance
train_positive = sum(1 for item in train_dataset if item['label'] == 1)
train_negative = sum(1 for item in train_dataset if item['label'] == 0)
print(f"Train - Positive: {train_positive}, Negative: {train_negative}")

Train - Positive: 9000, Negative: 9000


In [20]:
# Load tokenizer and model (DistilBERT for efficiency)
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [21]:
# Load pre-trained model for sentiment classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Training configuration
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.170000,0.182514
2,0.125500,0.206504
3,0.056800,0.249470


TrainOutput(global_step=3375, training_loss=0.13588959517302338, metrics={'train_runtime': 1294.8268, 'train_samples_per_second': 41.704, 'train_steps_per_second': 2.607, 'total_flos': 3576619763712000.0, 'train_loss': 0.13588959517302338, 'epoch': 3.0})

In [22]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json',
 './fine_tuned_distilbert/tokenizer.json')

In [24]:
import shutil
import os

# Create a zip file of the directory
shutil.make_archive('fine_tuned_distilbert', 'zip', './fine_tuned_distilbert')

# Download the zip file to your local computer
from google.colab import files
files.download('fine_tuned_distilbert.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.24947042763233185,
 'eval_runtime': 14.1925,
 'eval_samples_per_second': 140.92,
 'eval_steps_per_second': 8.807,
 'epoch': 3.0}

In [26]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [27]:
import evaluate
from datasets import load_dataset

# Load accuracy metric from evaluate library
accuracy_metric = evaluate.load("accuracy")

# Define the evaluation function
def compute_metrics(pred):
    # Predictions and labels
    predictions, labels = pred
    predictions = predictions.argmax(axis=-1)  # Get the predicted labels
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Set the compute_metrics function for the trainer
trainer.compute_metrics = compute_metrics

# Evaluate the model
evaluation_results = trainer.evaluate()

print(f"Evaluation Results: {evaluation_results}")

Evaluation Results: {'eval_loss': 0.24947042763233185, 'eval_accuracy': 0.9425, 'eval_runtime': 14.405, 'eval_samples_per_second': 138.841, 'eval_steps_per_second': 8.678, 'epoch': 3.0}


In [ ]:
## Save @ _Z_RegularProjects/generative-ai-with-azure-machine-learning/Implementing_Cloud_Based_NLP_with_Transformers

In [ ]:
# Create DataLoader for validation set
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to predict sentiment on validation set
def predict_validation_set(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)  # Get predicted class

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return predictions, true_labels

# Get predictions on test set
preds, true_labels = predict_validation_set(model, test_dataloader)

# Compute Accuracy
accuracy = np.mean(np.array(preds) == np.array(true_labels))
print(f"Validation Accuracy: {accuracy:.4f}")

# ==============================
# 🚀 Inference Function
# ==============================
def predict_sentiment(review_text):
    model.eval()
    inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding="max_length", max_length=256)

    # Move tensors to GPU if available
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        output = model(input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(output.logits, dim=1).cpu().item()

    sentiment = "Positive" if prediction == 1 else "Negative"
    return sentiment

# Example predictions
sample_reviews = [
    "This product is amazing! I love it.",
    "Worst purchase ever. Total waste of money.",
    "It's okay, not great but not bad."
]

for review in sample_reviews:
    print(f"Review: {review} | Sentiment: {predict_sentiment(review)}")